# Télécharger en bloc des données de WaPOR

Dans ce notebook, nous utiliserons un module local de Python pour automatiser les étapes de lecture des métadonnées, de requête d’url, de téléchargement d’ensemble de données raster, de pré-traitement d’ensemble de données raster en utilisant les informations de la métadonnée (tel que le facteur de conversion, et l’unité).

Exécuter la cellule code ci-dessous pour importer les packages et modules requis. Il vous sera demandé de fournir une clé d’API de WaPOR (voir la page [Start here](0_Start_here.ipynb) pour l’obtention de la clé d’API de WaPOR). Le module WaPOR se rappellera de votre clé et l’enregistrera dans le dossier du module. Si vous voulez utiliser une autre clé d’API, il vous faut supprimer ce fichier **'modules\WaPOR\wapor_api_token.pickle'**

In [1]:
import os
import glob
import sys
import shapefile             
import matplotlib.pyplot as plt 

folder=r"..\..\modules"
sys.path.append(folder) #ajouter un dossier avec les modules locaux aux chemins du système
import WaPOR #importer le  module local 'WaPOR'
WaPOR.API.version=2 #selectionner la version de WaPOR
WaPOR.API.getCatalog()

Loading WaPOR catalog...


,code,caption,description,additionalInfo,tags,workspaceCode,dataType,index,operation,hidden,links,measure,dimension
0,L1_GBWP_A,Gross Biomass Water Productivity,The annual Gross Biomass Water Productivity ex...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 1, 'context': 'LEVELS...",WAPOR_2,RASTER,101,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
1,L1_NBWP_A,Net Biomass Water Productivity,The annual Net Biomass Water Productivity expr...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 2, 'context': 'LEVELS...",WAPOR_2,RASTER,102,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
2,L1_AETI_A,Actual EvapoTranspiration and Interception (An...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 3, 'context': 'LEVELS...",WAPOR_2,RASTER,103,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
3,L1_AETI_M,Actual EvapoTranspiration and Interception (Mo...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 4, 'context': 'LEVELS...",WAPOR_2,RASTER,104,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'MONTH', 'caption': 'Month', 'worksp..."
4,L1_AETI_D,Actual EvapoTranspiration and Interception (De...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 5, 'context': 'LEVELS...",WAPOR_2,RASTER,105,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,L2_BAS_PHE_S,"Phenology (Seasonal, clipped by basin)","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4151,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'PHE', 'caption': 'Phenology', 'descr...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
192,L2_BAS_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4161,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
193,L2_CTY_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4162,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
194,L2_BAS_QUAL_LST_D,"Quality Land Surface Temperature (Dekadal, cli...","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4171,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."


## Code de l’ensemble de données

Le module **WaPOR** contient les fonctions *download_dekadal*, *download_monthly*, et *download_yearly* qui téléchargent en bloc les données de WaPOR à la résolution temporelle décadaire, mensuelle, et annuelle. Pour spécifier la couche de données à télécharger, ces fonctions requièrent le code de l’ensemble de données, les coordonnées des limites de l’encadré, le niveau des données, et les dates de début et fin de la période. Le code de chaque ensemble de données est disponible dans le tableau ci-dessous :  

| code        | Name          | 
| :---------: |:-------------:| 
| LCC | Classification de l’occupation des terres      | 
| PCP      | Précipitation      | 
| RET      | Evapotranspiration de référence      | 
| NPP      | Production primaire nette |
| AETI      | Evapotranspiration réelle et Interception |   
| E | Evaporation      |  
| I | Interception      | 
| T | Transpiration      | 
| PHE | Phenology      | 
| GBWP | Gross Biomass Water Productivity      | 

Utiliser le code des données sélectionnées plutôt que **RET** pour la valeur du paramètre **data** dans le code ci-dessous.

## Exemple de fonctions du module WaPOR

L’exemple de la cellule code ci-dessous montre comment le module *WaPOR* a été utilisé pour télécharger en bloc des données de WaPOR : l’Evapotranspiration de référence décadaire, mensuelle, et annuelle d’une étendue. L’étendue est définie par les limites de l’encadré du shapefile du bassin de Awash. Les données raster seront collectées et prétraitées à la correcte unité (respectivement, mm/décade, mm/mois, ou mm/an) et enregistrées dans [le dossier de sortie](data)

### Définir l’étendu de téléchargement et le dossier de sortie

Changer les valeurs de xmin, ymin, xmax, ymax à l’étendue de téléchargement. Si nécessaire, changer le chemin de *output_dir* au dossier où vous voulez enregistrer les données.

In [2]:
#definir l’étendue de téléchargement à partir d’un shapefile:
shape_fh=r".\data\Awash_shapefile.shp"
shape=shapefile.Reader(shape_fh)
xmin,ymin,xmax,ymax=shape.bbox

#définir l’étendue de téléchargement par des coordonnées:
# xmin,ymin,xmax,ymax=(32.65692516077674,-25.16412729789142,32.90420244933813,-25.01450956754679)

output_dir=r'.\data' #dossier pour enregistrer les données

### Télécharger des données journalières

Noter que les données journalières sont uniquement disponibles pour l’Evaporation de référence et la Précipitation au niveau 1

In [3]:
WaPOR.download_daily(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-01-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 dekadal RET data for the period 2009-01-01 till 2009-01-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_daily_L1_RET_E'

### Télécharger des données décadaires

In [4]:
WaPOR.download_dekadal(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 dekadal RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_dekadal_L1_RET_D'

### Télécharger des données mensuelles

In [5]:
WaPOR.download_monthly(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31', 
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 monthly RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_monthly_L1_RET_M'

### Télécharger des données annuelles

In [6]:
WaPOR.download_yearly(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 yearly RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_yearly_L1_RET_A'

### Télécharger des données saisonnières

**Remarques**: les données saisonnières ne sont disponibles que pour certaines couches.

In [ ]:
WaPOR.download_seasonal(output_dir,
                       data='GBWP',
                       Startdate='2009-01-01',
                       Enddate='2019-12-31',
                       latlim=[ymin-0.5, ymax+0.5],
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=2,
                       )
WaPOR.download_seasonal(output_dir,
                       data='PHE',
                       Startdate='2009-01-01',
                       Enddate='2019-12-31',
                       latlim=[ymin-0.5, ymax+0.5],
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=2,
                       )

# Exercice

Télécharger la Précipitation, l’Evapotranspiration réelle et l’Interception mensuelles et la classification de l’utilisation des terres annuelle de 2009 à 2010 pour la zone délimitée par le shapefile ".\data\Awash_shapefile.shp"

Utiliser les fonctions du module **WaPOR** suivant l’exemple donné.


In [7]:
'''
Écrivez votre code ici
'''

'\nÉcrivez votre code ici\n'